This is a simple Data Question-Answer (QA) bot that can fetch data from the relevant tables/files 
based on the user query by running a python query which is relevant to the question.
Note : this is a bot without memory

Pre-requisites : Mistral API Key

In [246]:
from langchain_mistralai.chat_models import ChatMistralAI
from langchain_core.tools import Tool, tool
from langchain_core.prompts import ChatPromptTemplate,PromptTemplate
from typing import Annotated, TypedDict
from langgraph.prebuilt import create_react_agent
from langchain_experimental.utilities import PythonREPL
from langgraph.graph import StateGraph,START,END
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
api_key = <your API key>

In [249]:
class State(TypedDict):
    question : Annotated[str, "The user question"]
    py_code : Annotated[str, "The python code"]
    result : Annotated[str, "The output from the python code"]
    answer : Annotated[str, "The final answer"]

In [250]:
def write_python(state:State)->str:

    """write python code """

    llm = ChatMistralAI(api_key=api_key, model_name='mistral-large-latest') 
    response = llm.invoke(input=[
        ("system", f"""Write a python code to fetch data from the relevant .csv file based on the given questiom.
                        Available csv files - [products.csv]. All column names are are in lowercase.
                        Return only the final python code as string that can be exceuted by Python Repl"""),
        ("human","what are the available products")
                                 ])
    return {"question" : state["question"], "py_code" : response.content, "result" : "", "answer" : ""}

In [251]:
def execute_python_code(state:State)->str:
    """Exceutes the python code and returns the result"""
    python_repl = PythonREPL()
    py_code = state["py_code"]
    return {"question": state["question"],  "py_code" : state[ "py_code"] ,"result":python_repl.run(py_code), "answer" : ""}

In [252]:
def generate(state:State)->str:
    """generate the final answer"""
    llm = ChatMistralAI(api_key=api_key, model_name='mistral-large-latest') 
    response = llm.invoke(input=[("system",f"""With the given question and result, generate the final answer. Always append the answer with 'FINAL ANSWER'
    {state["question"]}
    {state["result"]}""")])
    return {"question": state["question"],  "py_code" : state[ "py_code"] ,"result": state["result"], "answer" : response.content }


In [253]:
import uuid

thread_id = uuid.uuid4()
config = {"configurable":{"thread_id":thread_id}}
memory = MemorySaver()

In [254]:
workflow = StateGraph(State)
workflow.add_node("write_python",write_python)
workflow.add_node("execute_python_code", execute_python_code)
workflow.add_node("generate",generate)

In [255]:
workflow.add_edge(START,"write_python")
workflow.add_edge("write_python","execute_python_code")
workflow.add_edge("execute_python_code","generate")

In [256]:
graph = workflow.compile(checkpointer=memory, interrupt_before=["execute_python_code"])
graph.get_graph()

Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=RunnablePassthrough(), metadata=None), 'write_python': Node(id='write_python', name='write_python', data=write_python(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), 'execute_python_code': Node(id='execute_python_code', name='execute_python_code', data=execute_python_code(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata={'__interrupt': 'before'}), 'generate': Node(id='generate', name='generate', data=generate(tags=None, recurse=True, explode_args=False, func_accepts_config=False, func_accepts={}), metadata=None), '__end__': Node(id='__end__', name='__end__', data=None, metadata=None)}, edges=[Edge(source='__start__', target='write_python', data=None, conditional=False), Edge(source='execute_python_code', target='generate', data=None, conditional=False), Edge(source='write_python', target='execute_python_code', 

In [257]:
for step in graph.stream(
    {"question":"what are the available products"},
    config=config,
    stream_mode="updates"
):
    print(step)

try:
    user_approval = input("Do you want to go to execute query? (y/n): ")
except Exception:
    user_approval = "n"

if user_approval.lower() == "y":
    # If approved, continue the graph execution
    for step in graph.stream(None, config, stream_mode="updates"):
        print(step)
else:
    print("Operation cancelled by user.")

print(f'question : {step["generate"]["question"]}')
print(f'answer: {step["generate"]["answer"]}')

{'write_python': {'question': 'what are the available products', 'py_code': "```python\nimport pandas as pd\n\n# Load the products.csv file\ndata = pd.read_csv('products.csv')\n\n# Display the available products\navailable_products = data['product_name'].tolist()\nprint(available_products)\n```", 'result': '', 'answer': ''}}
{'__interrupt__': ()}
{'execute_python_code': {'question': 'what are the available products', 'py_code': "```python\nimport pandas as pd\n\n# Load the products.csv file\ndata = pd.read_csv('products.csv')\n\n# Display the available products\navailable_products = data['product_name'].tolist()\nprint(available_products)\n```", 'result': "['lakme red lipstick', 'lakme nude lipstick', 'lakme pink lipstick', 'lakme brown lipstick', 'huda red lipstick', 'huda nude lipstick', 'huda pink lipstick', 'huda brown lipstick', 'huda concealor-1', 'huda concealor-2', 'maybelline concealor-1', 'maybelline concealor-2']\n", 'answer': ''}}
{'generate': {'question': 'what are the ava

In [258]:
for step in graph.stream(
    {"question":"what are the available brands"},
    config=config,
    stream_mode="updates"
):
    print(step)

try:
    user_approval = input("Do you want to go to execute query? (y/n): ")
except Exception:
    user_approval = "n"

if user_approval.lower() == "y":
    # If approved, continue the graph execution
    for step in graph.stream(None, config, stream_mode="updates"):
        print(step)
else:
    print("Operation cancelled by user.")

print(f'question : {step["generate"]["question"]}')
print(f'answer: {step["generate"]["answer"]}')

{'write_python': {'question': 'what are the available brands', 'py_code': "```python\nimport csv\n\nwith open('products.csv', 'r') as file:\n    reader = csv.DictReader(file)\n    products = [row['product_name'] for row in reader]\n\nprint(products)\n```", 'result': '', 'answer': ''}}
{'__interrupt__': ()}
{'execute_python_code': {'question': 'what are the available brands', 'py_code': "```python\nimport csv\n\nwith open('products.csv', 'r') as file:\n    reader = csv.DictReader(file)\n    products = [row['product_name'] for row in reader]\n\nprint(products)\n```", 'result': "['lakme red lipstick', 'lakme nude lipstick', 'lakme pink lipstick', 'lakme brown lipstick', 'huda red lipstick', 'huda nude lipstick', 'huda pink lipstick', 'huda brown lipstick', 'huda concealor-1', 'huda concealor-2', 'maybelline concealor-1', 'maybelline concealor-2']\n", 'answer': ''}}
{'generate': {'question': 'what are the available brands', 'py_code': "```python\nimport csv\n\nwith open('products.csv', 'r'

In [259]:
for step in graph.stream(
    {"question":"what are the available categories"},
    config=config,
    stream_mode="updates"
):
    print(step)

try:
    user_approval = input("Do you want to go to execute query? (y/n): ")
except Exception:
    user_approval = "n"

if user_approval.lower() == "y":
    # If approved, continue the graph execution
    for step in graph.stream(None, config, stream_mode="updates"):
        print(step)
else:
    print("Operation cancelled by user.")

print(f'question : {step["generate"]["question"]}')
print(f'answer: {step["generate"]["answer"]}')

{'write_python': {'question': 'what are the available categories', 'py_code': "```python\nimport pandas as pd\n\n# Load the data from the CSV file\ndata = pd.read_csv('products.csv')\n\n# Assuming there is a column named 'product_name' that lists the available products\navailable_products = data['product_name'].tolist()\n\n# Print the available products\nprint(available_products)\n```", 'result': '', 'answer': ''}}
{'__interrupt__': ()}
{'execute_python_code': {'question': 'what are the available categories', 'py_code': "```python\nimport pandas as pd\n\n# Load the data from the CSV file\ndata = pd.read_csv('products.csv')\n\n# Assuming there is a column named 'product_name' that lists the available products\navailable_products = data['product_name'].tolist()\n\n# Print the available products\nprint(available_products)\n```", 'result': "['lakme red lipstick', 'lakme nude lipstick', 'lakme pink lipstick', 'lakme brown lipstick', 'huda red lipstick', 'huda nude lipstick', 'huda pink lips